In [48]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

for dirname, _, filenames in os.walk("/kaggle/input"):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/migros-datathon-coderspace/customeraccount.csv
/kaggle/input/migros-datathon-coderspace/sample_submission.csv
/kaggle/input/migros-datathon-coderspace/transaction_header.csv
/kaggle/input/migros-datathon-coderspace/genel_kategoriler.csv
/kaggle/input/migros-datathon-coderspace/customer.csv
/kaggle/input/migros-datathon-coderspace/train.csv
/kaggle/input/migros-datathon-coderspace/test.csv
/kaggle/input/migros-datathon-coderspace/product_groups.csv
/kaggle/input/migros-datathon-coderspace/transaction_sale/transaction_sale.csv


In [49]:
%pip install nb_black

import warnings
from tqdm import tqdm
from pathlib import Path
from itertools import repeat

import numpy as np
import pandas as pd
from scipy.stats import skew

from sklearn.metrics import f1_score
from sklearn.model_selection import KFold, StratifiedKFold, StratifiedGroupKFold
from sklearn.model_selection import train_test_split, cross_validate


from lightgbm import LGBMRegressor
import lightgbm as lgbm
from catboost import CatBoostClassifier, Pool
from catboost.utils import get_roc_curve, select_threshold

%load_ext lab_black
warnings.filterwarnings("ignore")

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.float_format", lambda x: "%.3f" % x)
pd.set_option("display.width", 500)

Note: you may need to restart the kernel to use updated packages.
The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [66]:
# submission_csv = pd.read_csv('../input/2022gdz22-datathon-case-2/submission.csv')
test_ = pd.read_csv("/kaggle/input/migros-datathon-coderspace/test.csv")
test = test_.copy()
train_ = pd.read_csv("/kaggle/input/migros-datathon-coderspace/train.csv")
train = train_.copy()


customer = pd.read_csv("/kaggle/input/migros-datathon-coderspace/customer.csv")
customer_account = pd.read_csv(
    "/kaggle/input/migros-datathon-coderspace/customeraccount.csv"
)
genel_kategoriler = pd.read_csv(
    "/kaggle/input/migros-datathon-coderspace/genel_kategoriler.csv"
)
product_groups = pd.read_csv(
    "/kaggle/input/migros-datathon-coderspace/product_groups.csv"
)
transaction_header = pd.read_csv(
    "/kaggle/input/migros-datathon-coderspace/transaction_header.csv"
)
transaction_sale = pd.read_csv(
    "/kaggle/input/migros-datathon-coderspace/transaction_sale/transaction_sale.csv"
)

In [51]:
test.nunique()

individualnumber    13236
category_number        49
hakkedis_amt          118
odul_amt               20
dtype: int64

In [52]:
train.nunique()

individualnumber    13115
category_number        49
hakkedis_amt          121
odul_amt               20
response                2
dtype: int64

In [105]:
samplesubmission.info()

NameError: name 'samplesubmission' is not defined

In [103]:
customer_account.nunique()

individualnumber    28593
cardnumber          35159
dtype: int64

In [73]:
customer.nunique()

individualnumber    28593
gender                  2
city_code              82
dateofbirth            87
dtype: int64

In [55]:
customer.head()

,individualnumber,gender,city_code,dateofbirth
0,94212124,K,NaN,1946.000
1,96387515,E,34.000,1974.000
2,95040383,E,80.000,1980.000
3,94694434,E,NaN,1970.000
4,47648671,E,35.000,1963.000


In [69]:
df_cust = pd.merge(customer_account, customer, on=["individualnumber"], how="inner")

In [70]:
df_cust.nunique()

individualnumber    28593
cardnumber          35159
gender                  2
city_code              82
dateofbirth            87
dtype: int64

In [56]:
train.head()

,individualnumber,category_number,hakkedis_amt,odul_amt,response
0,94230288,9000,21.000,2.000,0
1,4684087,9000,17.000,1.000,0
2,92472145,9058,24.000,3.000,0
3,88026681,9030,22.000,2.000,0
4,98127795,9001,38.000,3.000,0


In [75]:
genel_kategoriler.nunique()

category_number    50
genel_kategori      5
dtype: int64

In [58]:
product_groups.head()

,category_number,category_level_1,category_level_2,category_level_3,category_level_4
0,9046,341,20,10,10
1,9009,125,25,10,10
2,9049,410,65,20,10
3,9019,300,10,19,11
4,9035,305,31,10,10


In [59]:
transaction_header.head()

,date_of_transaction,cardnumber,basketid,is_sanal
0,2020-12-01,8487374630565615,20120163990041163,0
1,2020-12-01,4287574603569015,20120163990016137,0
2,2020-12-01,6487154648560315,20120160080017338,0
3,2020-12-01,3787564693563915,20120163570105022,0
4,2020-12-01,9087994683569715,20120162340016645,0


In [60]:
transaction_sale.head()

,basketid,category_level_1,category_level_2,category_level_3,category_level_4,amount,quantity,discount_type_1,discount_type_2,discount_type_3
0,20120706070177471,613,50,50,15,19.900,2.000,NaN,0.000,0.000
1,20120706070177471,425,30,12,10,3.500,1.000,NaN,0.000,0.000
2,20120706070177471,425,30,12,11,7.900,1.000,NaN,0.000,0.000
3,20122703940031323,103,25,15,10,7.900,2.000,NaN,0.000,0.000
4,20122703940031323,101,15,15,15,16.000,1.000,NaN,0.000,0.000


In [63]:
df_merge.head()

,basketid,category_level_1,category_level_2,category_level_3,category_level_4,amount,quantity,discount_type_1,discount_type_2,discount_type_3,date_of_transaction,cardnumber,is_sanal
0,20120706070177471,613,50,50,15,19.900,2.000,NaN,0.000,0.000,2020-12-07,6187974647569015,0
1,20120706070177471,425,30,12,10,3.500,1.000,NaN,0.000,0.000,2020-12-07,6187974647569015,0
2,20120706070177471,425,30,12,11,7.900,1.000,NaN,0.000,0.000,2020-12-07,6187974647569015,0
3,20120706070177471,131,20,15,30,78.470,0.982,NaN,9.820,0.000,2020-12-07,6187974647569015,0
4,20120706070177471,103,35,10,10,44.250,15.000,NaN,0.000,0.000,2020-12-07,6187974647569015,0


In [111]:
df_cust.nunique()

individualnumber    28593
cardnumber          35159
gender                  2
city_code              82
dateofbirth            87
dtype: int64

In [64]:
df_merge.nunique()

basketid               1124327
category_level_1            70
category_level_2            40
category_level_3            40
category_level_4            85
amount                   24700
quantity                  8380
discount_type_1        1557194
discount_type_2           5887
discount_type_3           6863
date_of_transaction        366
cardnumber               30316
is_sanal                     2
dtype: int64

In [77]:
df_trans = pd.merge(transaction_sale, transaction_header, on=["basketid"], how="inner")
# df1 = transaction_sale.append(df_merge)

# df1['Duplicated'] = df1.duplicated(keep=False) # keep=False marks the duplicated row with a True
##df_final = df1[~df1['Duplicated']] # selects only rows which are not duplicated.
# del df_final['Duplicated'] # delete the indicator column

In [78]:
df1 = pd.merge(df_trans, df_cust, on=["cardnumber"], how="inner")

In [113]:
df_whole = train.append(test, ignore_index=True)

In [115]:
df_whole.nunique()

individualnumber    26351
category_number        49
hakkedis_amt          126
odul_amt               20
response                2
dtype: int64

In [112]:
df1.nunique()

basketid               1124327
category_level_1            70
category_level_2            40
category_level_3            40
category_level_4            85
amount                   24700
quantity                  8380
discount_type_1        1557194
discount_type_2           5887
discount_type_3           6863
date_of_transaction        366
cardnumber               30316
is_sanal                     2
individualnumber         28576
gender                       2
city_code                   82
dateofbirth                 87
dtype: int64

In [83]:
train.nunique()

individualnumber    13115
category_number        49
hakkedis_amt          121
odul_amt               20
response                2
dtype: int64

In [82]:
test.nunique()

individualnumber    13236
category_number        49
hakkedis_amt          118
odul_amt               20
dtype: int64

In [84]:
df_train = pd.merge(train, df1, on=["individualnumber"], how="inner")

In [87]:
df_test = pd.merge(test, df1, on=["individualnumber"], how="inner")

individualnumber        13225
category_number            49
hakkedis_amt              118
odul_amt                   20
basketid               396205
category_level_1           70
category_level_2           40
category_level_3           39
category_level_4           82
amount                  17513
quantity                 5922
discount_type_1        454262
discount_type_2          3799
discount_type_3          4790
date_of_transaction       366
cardnumber              13764
is_sanal                    2
gender                      2
city_code                  81
dateofbirth                83
dtype: int64

In [86]:
df_train.nunique()

individualnumber        13111
category_number            49
hakkedis_amt              121
odul_amt                   20
response                    2
basketid               396642
category_level_1           67
category_level_2           39
category_level_3           39
category_level_4           83
amount                  17136
quantity                 5819
discount_type_1        452711
discount_type_2          3714
discount_type_3          4816
date_of_transaction       366
cardnumber              13667
is_sanal                    2
gender                      2
city_code                  82
dateofbirth                80
dtype: int64

In [80]:
df1.nunique()

basketid               1124327
category_level_1            70
category_level_2            40
category_level_3            40
category_level_4            85
amount                   24700
quantity                  8380
discount_type_1        1557194
discount_type_2           5887
discount_type_3           6863
date_of_transaction        366
cardnumber               30316
is_sanal                     2
individualnumber         28576
gender                       2
city_code                   82
dateofbirth                 87
dtype: int64

In [65]:
customer_account.head()

NameError: name 'customer_account' is not defined

In [ ]:
test.head()

In [ ]:
print("train obs. points:", train.shape[0])
print("test obs points:", test.shape[0])
test.shape[0] / (train.shape[0] + test.shape[0])

In [ ]:
# Column that existing in train but not in test.
[col for col in train.columns if col not in test.columns]

In [90]:
# Column that is existing in test but not in train.
[col for col in df_test.columns if col not in df_train.columns]

[]

In [91]:
y = df_train["response"]
X = df_train.drop(["response"], axis=1)
X_test = df_test

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.25, random_state=17
)  # 0.25 x 0.8 = 0.2

In [ ]:
datasets = {
    "x_train": X_train,
    "y_train": y_train,
    "x_val": X_val,
    "y_val": y_val,
    "x_test": X_test,
}
# X_train, X_test, y_train, y_test
def pred_model(model, **datasets):
    pred_val = model.predict(datasets["x_val"])
    pred_val = pd.DataFrame(pred_val, index=datasets["x_val"].index, columns=["pred"])

    pred_test = model.predict(datasets["x_test"])

    return pred_test, pred_val

In [ ]:
X_train.isnull().sum()

In [ ]:
lgbm_params = {
    "learning_rate": [0.01, 0.03],
    "colsample_bytree": [0.3, 0.7],
    "num_boost_round": [500],
}

In [ ]:
X.head()

In [ ]:
X.dtypes

In [ ]:
lgbm_model = LGBMRegressor(random_state=46)

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

lgbm_model = LGBMRegressor(random_state=46)
rmse = np.mean(
    np.sqrt(-cross_val_score(lgbm_model, X, y, cv=5, scoring="neg_mean_squared_error"))
)
rmse

In [ ]:
lgbm_params = {
    "learning_rate": [0.01, 0.03],
    "n_estimators": [500, 1000],
    "colsample_bytree": [0.3, 0.7],
}

lgbm_gs_best = GridSearchCV(lgbm_model, lgbm_params, cv=3, n_jobs=-1, verbose=True).fit(
    X, y
)

In [ ]:
lgbm_gs_best.best_params_
lgbm_final = lgbm_model.set_params(**lgbm_gs_best.best_params_, random_state=17).fit(
    X, y
)

In [ ]:
rmse = np.mean(
    np.sqrt(-cross_val_score(lgbm_final, X, y, cv=5, scoring="neg_mean_squared_error"))
)

In [ ]:
rmse

In [98]:
len(df_test.index)

2424171

In [100]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2388292 entries, 0 to 2388291
Data columns (total 21 columns):
 #   Column               Dtype  
---  ------               -----  
 0   individualnumber     int64  
 1   category_number      int64  
 2   hakkedis_amt         float64
 3   odul_amt             float64
 4   response             int64  
 5   basketid             int64  
 6   category_level_1     int64  
 7   category_level_2     int64  
 8   category_level_3     int64  
 9   category_level_4     int64  
 10  amount               float64
 11  quantity             float64
 12  discount_type_1      float64
 13  discount_type_2      float64
 14  discount_type_3      float64
 15  date_of_transaction  object 
 16  cardnumber           int64  
 17  is_sanal             int64  
 18  gender               object 
 19  city_code            float64
 20  dateofbirth          float64
dtypes: float64(9), int64(10), object(2)
memory usage: 400.9+ MB


In [101]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13115 entries, 0 to 13114
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   individualnumber  13115 non-null  int64  
 1   category_number   13115 non-null  int64  
 2   hakkedis_amt      13115 non-null  float64
 3   odul_amt          13115 non-null  float64
 4   response          13115 non-null  int64  
dtypes: float64(2), int64(3)
memory usage: 512.4 KB


In [ ]:

y_pred2 = lgbm_final.predict(X_test)

In [ ]:
y_pred2[y_pred2 < 0] = 0
y_pred2[y_pred2 < 0.0005] = 1
y_pred2 = [round(pred) for pred in y_pred2]

In [106]:
sample_submission = pd.read_csv(
    "/kaggle/input/migros-datathon-coderspace/sample_submission.csv"
)

sample_submission["response"] = y_pred2

In [109]:
sample_submission.head()

,individualnumber,response
0,95812936,NaN
1,23929570,NaN
2,95948115,NaN
3,100898513,NaN
4,12769156,NaN


In [ ]:
sample_submission.dtypes

In [ ]:
# you could use any filename. We choose submission here
sample_submission.to_csv("/kaggle/working/submission2.csv", index=False)

In [ ]:
lgbm_final = lgbm_model.set_params(**lgbm_params).fit(X_train,y_train)